In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

import xgboost as xgb
from sklearn.model_selection import train_test_split

param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
        }
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)
test['Attrition']=predict
# 转化为二分类输出
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test[['Attrition']].to_csv('submit_xgb.csv')


[0]	train-auc:0.847165	valid-auc:0.684547
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.
[25]	train-auc:0.962632	valid-auc:0.760941
[50]	train-auc:0.972215	valid-auc:0.77431
[75]	train-auc:0.986662	valid-auc:0.777693


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[100]	train-auc:0.991351	valid-auc:0.776056
[125]	train-auc:0.993668	valid-auc:0.780967
[150]	train-auc:0.995842	valid-auc:0.785551
[175]	train-auc:0.99762	valid-auc:0.78937
[200]	train-auc:0.998518	valid-auc:0.795154
[225]	train-auc:0.999003	valid-auc:0.800065
[250]	train-auc:0.999299	valid-auc:0.802139
[275]	train-auc:0.999677	valid-auc:0.801266
[300]	train-auc:0.999829	valid-auc:0.803994
[325]	train-auc:0.999919	valid-auc:0.803994
[350]	train-auc:0.999946	valid-auc:0.802794
[375]	train-auc:0.999973	valid-auc:0.805304
[400]	train-auc:0.999982	valid-auc:0.807596
[425]	train-auc:1	valid-auc:0.807814
[450]	train-auc:1	valid-auc:0.810215
[475]	train-auc:1	valid-auc:0.811306
[500]	train-auc:1	valid-auc:0.810761
[525]	train-auc:1	valid-auc:0.811306
[550]	train-auc:1	valid-auc:0.811961
[575]	train-auc:1	valid-auc:0.811415
[600]	train-auc:1	valid-auc:0.810761
[625]	train-auc:1	valid-auc:0.811634
[650]	train-auc:1	valid-auc:0.811743
[675]	train-auc:1	valid-auc:0.811852
[700]	train-auc:1	valid

In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

import catboost as cb
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

model = cb.CatBoostClassifier(iterations=1000, 
                              depth=7, 
                              learning_rate=0.01, 
                              loss_function='Logloss', 
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=50
                             )

# 得到分类特征的列号
categorical_features_indices = []
for i in range(len(X_train.columns)):
    if X_train.columns.values[i] in attr:
        categorical_features_indices.append(i)
print(categorical_features_indices)

model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=categorical_features_indices)

#model = cb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test)
#predict = model.predict_proba(test)
#print(predict)
test['Attrition']=predict
## 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test[['Attrition']].to_csv('submit_cb.csv')


[0, 1, 3, 5, 6, 9, 13, 15, 19, 20]
0:	test: 0.6390374	best: 0.6390374 (0)	total: 61.4ms	remaining: 1m 1s
50:	test: 0.7998472	best: 0.7998472 (50)	total: 413ms	remaining: 7.68s
100:	test: 0.8054131	best: 0.8054131 (100)	total: 716ms	remaining: 6.38s
150:	test: 0.8053039	best: 0.8054131 (100)	total: 1.12s	remaining: 6.32s
200:	test: 0.8075958	best: 0.8075958 (200)	total: 1.55s	remaining: 6.16s
250:	test: 0.8062862	best: 0.8075958 (200)	total: 2s	remaining: 5.97s
300:	test: 0.8045400	best: 0.8075958 (200)	total: 2.4s	remaining: 5.57s
350:	test: 0.8059587	best: 0.8075958 (200)	total: 2.78s	remaining: 5.13s
400:	test: 0.8065044	best: 0.8075958 (200)	total: 3.18s	remaining: 4.74s
450:	test: 0.8065044	best: 0.8075958 (200)	total: 3.53s	remaining: 4.3s
500:	test: 0.8077049	best: 0.8077049 (500)	total: 3.92s	remaining: 3.91s
550:	test: 0.8090145	best: 0.8090145 (550)	total: 4.28s	remaining: 3.49s
600:	test: 0.8106515	best: 0.8106515 (600)	total: 4.64s	remaining: 3.08s
650:	test: 0.8113063	best:

In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

import lightgbm as lgb
from sklearn.model_selection import train_test_split
# param = {
#     'num_leaves':41,
#     'boosting_type': 'gbdt',
#     'objective':'binary',
#     'max_depth':15,
#     'learning_rate':0.001,
#     'metric':'binary_logloss'}
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', #
                         #'metric' : 'binary_logloss',
                         'metric' : 'auc',
#                          'metric' : 'self_metric',
                         'learning_rate' : 0.01,
                         'max_depth' : 15,
                         'feature_fraction':0.8,
                         'bagging_fraction': 0.9,
                         'bagging_freq': 8,
                         'lambda_l1': 0.6,
                         'lambda_l2': 0,
#                          'scale_pos_weight':k,
#                         'is_unbalance':True
        }
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)


model = lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round = 10000 ,early_stopping_rounds=200,verbose_eval=25, categorical_feature=attr)
predict=model.predict(test)
#print(predict)
test['Attrition']=predict
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
test[['Attrition']].to_csv('submit_lgb.csv')



OSError: dlopen(/opt/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /opt/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

import ngboost as ng
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

model = ng.NGBClassifier(n_estimators=1000, 
                              learning_rate=0.01, 
                              verbose=True, 
                              verbose_eval=50
                             )


model.fit(X_train, y_train)

#predict = model.predict(test)
predict = model.predict_proba(test)[:, 1]
#print(predict)
test['Attrition']=predict
## 转化为二分类输出
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test[['Attrition']].to_csv('submit_ngb.csv')


[iter 0] loss=0.4190 val_loss=0.0000 scale=1.0000 norm=2.0000
[iter 50] loss=0.3427 val_loss=0.0000 scale=1.0000 norm=1.7212
[iter 100] loss=0.3005 val_loss=0.0000 scale=1.0000 norm=1.6308
[iter 150] loss=0.2782 val_loss=0.0000 scale=1.0000 norm=1.5989
[iter 200] loss=0.2683 val_loss=0.0000 scale=1.0000 norm=1.5975
[iter 250] loss=0.2606 val_loss=0.0000 scale=1.0000 norm=1.5968
[iter 300] loss=0.2531 val_loss=0.0000 scale=1.0000 norm=1.5853
[iter 350] loss=0.2493 val_loss=0.0000 scale=1.0000 norm=1.5803
[iter 400] loss=0.2465 val_loss=0.0000 scale=0.5000 norm=0.7870
[iter 450] loss=0.2442 val_loss=0.0000 scale=0.1250 norm=0.1966
[iter 500] loss=0.2421 val_loss=0.0000 scale=1.0000 norm=1.5711
[iter 550] loss=0.2407 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 600] loss=0.2407 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 650] loss=0.2407 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 700] loss=0.2407 val_loss=0.0000 scale=0.0010 norm=0.0015
[iter 750] loss=0.2402 val_loss=0.0000 scal

In [2]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=test1=pd.read_csv('test.csv',index_col=0)

#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)


mms = MinMaxScaler(feature_range=(0, 1))
X_train = mms.fit_transform(X_train)
X_valid = mms.fit_transform(X_valid)
test = mms.fit_transform(test)

model = SVC(kernel='rbf', 
			gamma="auto",
			max_iter=1000,
			random_state=33,
			verbose=True,
			tol=1e-5,
			cache_size=50000
		   )
#print(X_train)
#print(y_train)
#print(sum(y_train))

model = LinearSVC(
			max_iter=1000,
			random_state=33,
			verbose=True,
		   )
model.fit(X_train, y_train)
predict = model.predict(test)
print(predict)
#print(test)
#predict = model.predict_proba(test)[:, 1]
test1['Attrition']=predict

# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test1[['Attrition']].to_csv('submit_svc.csv')



[LibLinear][0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
